In [6]:
import sympy as sp
import numpy as np

In [7]:
def input_metric_tensor(n):
    # Get user-defined variable names and create symbols
    variables = []
    for i in range(n):
        variable = input(f"Enter variable {i + 1} (e.g., x, y, α, β): ")
        variables.append(sp.symbols(variable))

    # Initialize an empty list to hold the elements of the metric tensor
    metric_components = []

    # Prompt the user for each component of the n x n metric tensor
    print(f"Enter the components of the {n}x{n} metric tensor g (g_ij):")
    for i in range(n):
        for j in range(n):
            value = input(f"g_{i+1}{j+1} (component at row {i+1}, column {j+1}): ")
            # Convert the input string into a symbolic expression in terms of the chosen variables
            metric_components.append(sp.sympify(value))  # Using sympify to automatically recognize the variables

    # Create the n x n metric tensor using the input components
    g = sp.Matrix(n, n, metric_components)
    return g, variables

In [8]:
def calculate_inverse(matrix):
    try:
        # Attempt to compute the inverse of the matrix
        inverse_matrix = matrix.inv()
        print("The inverse of the matrix g is:")
        sp.pprint(inverse_matrix)
        return inverse_matrix
    except ValueError as e:
        # Catch and print an error if the matrix is singular
        print("Error: The matrix is singular and cannot be inverted.")
        print(f"Details: {e}")
        return None

## christoffel symbols equations
$$\Gamma^{k}_{ij} = \frac{1}{2} g^{kl} \left( \frac{\partial g_{jl}}{\partial x^i} + \frac{\partial g_{il}}{\partial x^j} - \frac{\partial g_{ij}}{\partial x^l} \right)$$ 

In [9]:
def calculate_christoffel_symbols(g, g_inv, variables, print_flag=False):
    n = g.shape[0]  # Get the size of the tensor

    # Create empty matrices for the Christoffel symbols
    gamma = [sp.Matrix(n, n, [0] * (n * n)) for _ in range(n)]  # Γ^k_ij
    #sp.pprint(gamma)
    #sp.pprint(gamma[0])  # Print the first Christoffel symbol for debugging

    # Calculate Christoffel symbols
    for k in range(n):
        for i in range(n):
            for j in range(n):
                '''
                term = sum(sp.diff(g[i, l], variables[j]) + sp.diff(g[j, l], variables[i]) - sp.diff(g[i, j], variables[l]) for l in range(n))

                # Assign the formula for the Christoffel symbol
                gamma[k][i][j] = sp.Rational(1, 2) * sum(g_inv[k, l] * term for l in range(n))
                '''
                term = sp.Matrix(n, n, [0] * (n * n))
                for l in range(n):
                    term[l] = sp.diff(g[i, l], variables[j]) + sp.diff(g[j, l], variables[i]) - sp.diff(g[i, j], variables[l])
                # Assign the formula for the Christoffel symbol
                gamma[k][i,j] = sp.Rational(1, 2) * sum(g_inv[k, l] * term[l] for l in range(n))
                

    return gamma

## test

In [10]:
n = 2
g, variables = input_metric_tensor(n)
print("variables:")
sp.pprint(variables)
print("\ng:")
sp.pprint(g)

#calculate the inverse metric tensor
g_inv = calculate_inverse(g)
print("\ng_inv:")
print(g_inv)

# Calculate the Christoffel symbols
if g_inv is not None:
    gamma = calculate_christoffel_symbols(g, g_inv, variables, print_flag=True)
    
    # Print the Christoffel symbols
    print("\nChristoffel symbols (Γ^k_ij):")
    for k in range(n):
        print(f"Γ^{k+1}_ij:")
        sp.pprint(gamma[k])
        print()
else:
    print("Cannot calculate Christoffel symbols due to singular metric tensor.")

Enter the components of the 2x2 metric tensor g (g_ij):
variables:
[x, y]

g:
⎡1  0⎤
⎢    ⎥
⎣0  y⎦
The inverse of the matrix g is:
⎡1  0⎤
⎢    ⎥
⎢   1⎥
⎢0  ─⎥
⎣   y⎦

g_inv:
Matrix([[1, 0], [0, 1/y]])

Christoffel symbols (Γ^k_ij):
Γ^1_ij:
⎡0  0⎤
⎢    ⎥
⎣0  0⎦

Γ^2_ij:
⎡0   0 ⎤
⎢      ⎥
⎢    1 ⎥
⎢0  ───⎥
⎣   2⋅y⎦

